In [1]:
import time
import numpy as np

from tssearch.search.query_search import time_series_search

In [2]:
number_of_repetitions = 30

* The currently tested packages can only work with distances that do not allow the computation of data of different nature. For this reason, a public data set obtained from https://finance.yahoo.com/ has been used. All data are continuous.

In [3]:
# For Multivariate Time Series
x = np.load('../../../Data/E2_FinanceData/FinanceData_20days.npy', allow_pickle=True)

print(x.shape)

time_dtw_dep = []

dict_distances = {
    "elastic": {"Dynamic Time Warping": {
        "multivariate": "yes",
        "description": "",
        "function": "dtw",
        "parameters": {"dtw_type": "dtw", "alpha": 1},
        "use": "yes"}
    }
}

for rep in range(number_of_repetitions):
    
    inicio = time.time()
    dtw_matrix = np.zeros((len(x), len(x)))
    for i in range(len(x)):
        for j in range(len(x)):
            pat1 = x[i]
            pat2 = x[j]
            result = time_series_search(dict_distances, pat1, pat2, output=("number", 1))
            dtw_matrix[i,j] = result

    fin = time.time()
    time_dtw_dep.append(fin-inicio)
    
# f = open("../../results_E2_SyntheticData/pyts.txt", "a")
# f.write(str(time_dtw_dep))
# f.close()

(505, 20, 6)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 2d, C)[0m
[0m[1mDuring: typing of argument at C:\Users\escud\anaconda3\envs\phd_work\lib\site-packages\tssearch\distances\elastic_utils.py (50)[0m
[1m
File "..\..\..\..\..\..\..\anaconda3\envs\phd_work\lib\site-packages\tssearch\distances\elastic_utils.py", line 50:[0m
[1mdef _multidimensional_cost_matrix(x, y, weight):
    <source elided>
    """
[1m    l1 = x.shape[0]
[0m    [1m^[0m[0m


In [6]:
x = np.random.rand(5,5)
y = np.random.rand(5,5)
time_series_search(dict_distances, x, y, output=("number", 1))

{'Dynamic Time Warping': {'path_dist': array([3.38058257]),
  'distance': array([4.48528039, 4.84664237, 3.38058257, 3.83961117, 4.2221365 ]),
  'start': [1],
  'end': array([2], dtype=int64),
  'path': [(array([0, 1, 2, 3], dtype=int64),
    array([1, 1, 1, 1], dtype=int64))]}}

It does not allow working with multivariate time series (MTS). In case of entering a MTS it treats it with UTS and returns the distance between UTS. 